# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

## Scrapping Data From Wikipedia

In [2]:
#website we want to get de html
website = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Get the html from the website
source = requests.get(website).text

In [3]:
soup = BeautifulSoup(source,'lxml')
#get the table
table = soup.find('table',{'class':'wikitable sortable'})
#get the content of the table
content = table.get_text().split('\n')

In [4]:
content[0:10]

['',
 '',
 'Postcode',
 'Borough',
 'Neighbourhood',
 '',
 '',
 'M1A',
 'Not assigned',
 'Not assigned']

In [5]:
#remove empty values from the list
content = list(filter(None,content))

In [6]:
content[0:10]

['Postcode',
 'Borough',
 'Neighbourhood',
 'M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A']

In [7]:
postcode = []
borough = []
neighborhood = []
cont = 0

#create the vectors of the three columns
for i in range(len(content)):
    if cont == 0:
        postcode.append(content[i])
        cont = cont + 1
    elif cont == 1:
        borough.append(content[i])
        cont = cont + 1
    else:
        neighborhood.append(content[i])
        cont = 0

In [8]:
print(postcode[0:5])
print(borough[0:5])
print(neighborhood[0:5])

['Postcode', 'M1A', 'M2A', 'M3A', 'M4A']
['Borough', 'Not assigned', 'Not assigned', 'North York', 'North York']
['Neighbourhood', 'Not assigned', 'Not assigned', 'Parkwoods', 'Victoria Village']


In [9]:
#create the dataframe
df = pd.DataFrame()
df[postcode[0]] = postcode[1:]
df[borough[0]] = borough[1:]
df[neighborhood[0]] = neighborhood[1:]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [10]:
#correct the column title
df.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace = True)

In [11]:
#Remove the 'Not assigned' data
df.drop(index = df[(df['Neighborhood'] == 'Not assigned') & (df['Borough'] == 'Not assigned')].index, inplace = True)

In [12]:
#reset the index
df.reset_index(drop = True, inplace = True)

In [13]:
#check if still has 'Not assigned' data
df[df['Borough'] == 'Not assigned']

,Postcode,Borough,Neighborhood


In [14]:
df[df['Neighborhood'] == 'Not assigned']

,Postcode,Borough,Neighborhood
6,M7A,Queen's Park,Not assigned


In [15]:
df.Neighborhood.replace('Not assigned',"Queen's Park", inplace = True)

In [16]:
df[df['Neighborhood'] == 'Not assigned']

,Postcode,Borough,Neighborhood


In [17]:
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [18]:
#group the neighborhoods by the poscode
df_final = df.groupby('Postcode')['Neighborhood'].apply(','.join).reset_index()

In [19]:
#add the 'Borough' column
df_final = df_final.merge(df[['Postcode','Borough']],on = 'Postcode', how = 'inner').drop_duplicates()
df_final.reset_index(inplace = True, drop = True)

In [20]:
#put the columns in the right sequence
df_final = df_final[['Postcode','Borough','Neighborhood']]
df_final.sort_values(by = 'Postcode', inplace = True)

In [21]:
df_final.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Getting the Geographical Coordinates of the Neighborhoods

In [22]:
web_coord = 'http://cocl.us/Geospatial_data'
coord = pd.read_csv(web_coord)

In [23]:
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
#rename the column 'Postal Code', so we can merge coord with the df_final
coord.rename(columns = {'Postal Code':'Postcode'}, inplace = True)
coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
df_coord = df_final.merge(coord, on = 'Postcode')
df_coord.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [26]:
df_coord.shape

(103, 5)

## Clustering Neighborhoods

In [27]:
import folium
import json
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

In [45]:
#get the geographical coordinates of Toronto
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with the neighborhoods 

In [46]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

#add markers to the map

for lat, lng, postcode, borough, neighborhood in zip(df_coord['Latitude'], df_coord['Longitude'], df_coord['Postcode'],df_coord['Borough'], df_coord['Neighborhood']):
    label = '{} - {} - {}'.format(postcode, borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [47]:
CLIENT_ID = 'IXYUHV2CKMPMLB00RL144243ALVWMGDLJRUJQDFJUV1O3TIL' # your Foursquare ID
CLIENT_SECRET = 'FO13DDOIJ3Y0UYH3DRTARW3JAGOVFYFHUPR0PO1W2DOIZQ04' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IXYUHV2CKMPMLB00RL144243ALVWMGDLJRUJQDFJUV1O3TIL
CLIENT_SECRET:FO13DDOIJ3Y0UYH3DRTARW3JAGOVFYFHUPR0PO1W2DOIZQ04


### Explore Neighborhoods in Toronto

Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
#creat a dataframe 
toronto_venues = getNearbyVenues(names = df_coord['Neighborhood'],
                                   latitudes = df_coord['Latitude'],
                                   longitudes = df_coord['Longitude']
                                  )


In [50]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [51]:
print('Number of differents types of venues:',len(toronto_venues['Venue Category'].unique()))

Number of differents types of venues: 276


In [52]:
toronto_venues.shape

(2263, 7)

In [53]:
category = pd.get_dummies(toronto_venues[['Venue Category']], prefix = ' ', prefix_sep = '')

# add neighborhood column back to dataframe
toronto_category = pd.concat([toronto_venues['Neighborhood'], category], axis = 1)

toronto_category.head()


,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [54]:
toronto_grouped = toronto_category.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,...,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [55]:
toronto_grouped.shape

(100, 277)

Let's create a dataframe with the top 10 most common venues in a neighborhood

In [56]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,American Restaurant,Hotel,Restaurant,Burger Joint,Asian Restaurant
1,Agincourt,Lounge,Skating Rink,Breakfast Spot,Clothing Store,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Coffee Shop,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fast Food Restaurant,Liquor Store,Japanese Restaurant,Discount Store,Sandwich Place,Beer Store,Pizza Place,Pharmacy,Fried Chicken Joint
4,"Alderwood,Long Branch",Pizza Place,Sandwich Place,Gym,Skating Rink,Dance Studio,Pharmacy,Coffee Shop,Athletics & Sports,Pub,Yoga Studio


### Cluster Neighborhoods

Run *k*-means to divide the neighborhoods in 5 clusters

In [58]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0])

Let's create a dataframe with the information of the neighborhood as well the top 10 most common venues and the cluster

In [59]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_coord

# merge toronto_grouped with df_coord to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3.0,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Field
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0.0,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Diner
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Pizza Place,Rental Car Location,Breakfast Spot,Electronics Store,Medical Center,Intersection,Mexican Restaurant,Yoga Studio,Doner Restaurant,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Bakery,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Fried Chicken Joint,Yoga Studio,Donut Shop,Dog Run


In [63]:
toronto_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 16 columns):
Postcode                  103 non-null object
Borough                   103 non-null object
Neighborhood              103 non-null object
Latitude                  103 non-null float64
Longitude                 103 non-null float64
Cluster Labels            100 non-null float64
1st Most Common Venue     100 non-null object
2nd Most Common Venue     100 non-null object
3rd Most Common Venue     100 non-null object
4th Most Common Venue     100 non-null object
5th Most Common Venue     100 non-null object
6th Most Common Venue     100 non-null object
7th Most Common Venue     100 non-null object
8th Most Common Venue     100 non-null object
9th Most Common Venue     100 non-null object
10th Most Common Venue    100 non-null object
dtypes: float64(3), object(13)
memory usage: 13.7+ KB


In [67]:
#check for NaN values
toronto_merged[np.isnan(toronto_merged['Cluster Labels']) == True]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,M2M,North York,"Newtonbrook,Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As we can see, there are three neighborhoods that we don't have information, so let's drop them

In [68]:
toronto_merged.dropna(inplace = True)

In [70]:
toronto_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 102
Data columns (total 16 columns):
Postcode                  100 non-null object
Borough                   100 non-null object
Neighborhood              100 non-null object
Latitude                  100 non-null float64
Longitude                 100 non-null float64
Cluster Labels            100 non-null float64
1st Most Common Venue     100 non-null object
2nd Most Common Venue     100 non-null object
3rd Most Common Venue     100 non-null object
4th Most Common Venue     100 non-null object
5th Most Common Venue     100 non-null object
6th Most Common Venue     100 non-null object
7th Most Common Venue     100 non-null object
8th Most Common Venue     100 non-null object
9th Most Common Venue     100 non-null object
10th Most Common Venue    100 non-null object
dtypes: float64(3), object(13)
memory usage: 13.3+ KB


Let's visualize the clusters

In [71]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters